# Debugging and Profiling

**Chapter 10 - Learning Python, 5th Edition**

Effective debugging and performance analysis are essential professional skills.
Python provides built-in tools ranging from simple `print` debugging to the
`pdb` debugger, `timeit` for micro-benchmarks, and `cProfile` for profiling.
Understanding common performance patterns helps you write efficient code
from the start.

## Print Debugging and the logging Module

While `print()` is the simplest debugging tool, the `logging` module provides
configurable, leveled output that can be left in production code. Logging levels
(DEBUG, INFO, WARNING, ERROR, CRITICAL) let you control verbosity without
changing code.

In [ ]:
import logging
import sys


# --- Print debugging: useful but limited ---
def find_first_duplicate(items: list[int]) -> int | None:
    """Find the first duplicate in a list (with print debugging)."""
    seen: set[int] = set()
    for i, item in enumerate(items):
        print(f"  [DEBUG] i={i}, item={item}, seen={seen}")  # Debug output
        if item in seen:
            print(f"  [DEBUG] Found duplicate: {item} at index {i}")
            return item
        seen.add(item)
    return None


print("=== Print debugging ===")
result = find_first_duplicate([3, 1, 4, 1, 5])
print(f"  Result: {result}")


# --- Logging: professional, configurable debugging ---
# Reset any existing handlers for clean demo output
root_logger = logging.getLogger()
for handler in root_logger.handlers[:]:
    root_logger.removeHandler(handler)

# Configure logging with a structured format
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)-8s] %(name)s: %(message)s",
    datefmt="%H:%M:%S",
    stream=sys.stdout,
)

# Create a module-specific logger
logger = logging.getLogger("myapp.processor")


def process_batch(items: list[str]) -> list[str]:
    """Process a batch of items with proper logging."""
    logger.info("Starting batch processing of %d items", len(items))
    results: list[str] = []

    for i, item in enumerate(items):
        logger.debug("Processing item %d: %r", i, item)
        if not item.strip():
            logger.warning("Skipping empty item at index %d", i)
            continue
        try:
            processed = item.strip().upper()
            results.append(processed)
        except Exception as e:
            logger.error("Failed to process item %d: %s", i, e, exc_info=True)

    logger.info("Batch complete: %d/%d items processed", len(results), len(items))
    return results


print("\n=== Logging module ===")
results = process_batch(["  hello ", "", "world  ", "  python"])
print(f"\n  Results: {results}")

# Changing log level filters output without code changes
print("\n=== With WARNING level (less verbose) ===")
logger.setLevel(logging.WARNING)
results = process_batch(["test", "", "data"])
print(f"  Results: {results}")

## The pdb Debugger

Python's built-in debugger `pdb` lets you pause execution and inspect state
interactively. Since Python 3.7, `breakpoint()` is the preferred way to
enter the debugger.

Essential pdb commands:

| Command | Shortcut | Description |
|---|---|---|
| `next` | `n` | Execute next line (step OVER functions) |
| `step` | `s` | Step INTO function calls |
| `continue` | `c` | Continue until next breakpoint |
| `print expr` | `p expr` | Print the value of an expression |
| `list` | `l` | Show source code around current line |
| `where` | `w` | Show the call stack (traceback) |
| `up` / `down` | `u` / `d` | Move up/down the call stack |
| `break` | `b` | Set a breakpoint |
| `quit` | `q` | Quit the debugger |

**Note**: pdb is interactive and does not work well in notebooks. The code
below shows how you would use it in a normal Python script.

In [ ]:
# pdb usage examples (shown as reference; pdb is interactive, not for notebooks)

pdb_example_basic = '''
# === Method 1: breakpoint() (Python 3.7+, preferred) ===
def calculate_discount(price: float, discount_pct: float) -> float:
    """Calculate discounted price."""
    breakpoint()  # Execution pauses here; opens pdb prompt
    discount = price * (discount_pct / 100)
    final_price = price - discount
    return final_price

# === Method 2: import pdb; pdb.set_trace() (older style) ===
import pdb

def process_data(items: list[int]) -> list[int]:
    results = []
    for item in items:
        if item < 0:
            pdb.set_trace()  # Stop here to investigate negative values
        results.append(item * 2)
    return results
'''

pdb_session_example = '''
# === Example pdb session ===
$ python my_script.py
> /path/to/my_script.py(4)calculate_discount()
-> discount = price * (discount_pct / 100)

(Pdb) p price            # Print variable value
100.0
(Pdb) p discount_pct
15.0
(Pdb) n                  # Execute next line
> /path/to/my_script.py(5)calculate_discount()
-> final_price = price - discount
(Pdb) p discount         # Check computed value
15.0
(Pdb) l                  # List source code
  1     def calculate_discount(price: float, discount_pct: float) -> float:
  2         breakpoint()
  3         discount = price * (discount_pct / 100)
  4  ->     final_price = price - discount
  5         return final_price
(Pdb) w                  # Where: show call stack
  /path/to/my_script.py(8)<module>()
-> result = calculate_discount(100.0, 15.0)
> /path/to/my_script.py(5)calculate_discount()
-> final_price = price - discount
(Pdb) c                  # Continue execution
'''

print("=== pdb usage patterns ===")
print(pdb_example_basic)
print("=== Example interactive session ===")
print(pdb_session_example)

# Tip: disable all breakpoints without removing them
print("=== Useful environment variables ===")
tips = {
    "PYTHONBREAKPOINT=0": "Disable all breakpoint() calls (for production)",
    "PYTHONBREAKPOINT=ipdb.set_trace": "Use ipdb (enhanced debugger) instead of pdb",
    "PYTHONBREAKPOINT=pudb.set_trace": "Use pudb (TUI debugger) instead of pdb",
}
for var, desc in tips.items():
    print(f"  {var}")
    print(f"    -> {desc}")

## Assert Statements for Invariants

`assert` statements verify conditions that should always be true.
They serve as executable documentation and catch bugs early during
development. Assertions can be globally disabled with `python -O`,
so never use them for input validation or error handling.

In [ ]:
from dataclasses import dataclass


@dataclass
class BankAccount:
    """Bank account with assertion-guarded invariants."""

    owner: str
    _balance: float = 0.0

    def __post_init__(self) -> None:
        assert isinstance(self.owner, str) and self.owner, "Owner must be a non-empty string"
        assert self._balance >= 0, f"Initial balance cannot be negative: {self._balance}"

    @property
    def balance(self) -> float:
        return self._balance

    def deposit(self, amount: float) -> None:
        # Precondition: amount must be positive
        assert amount > 0, f"Deposit amount must be positive, got {amount}"

        old_balance = self._balance
        self._balance += amount

        # Postcondition: balance increased by exactly the deposit amount
        assert self._balance == old_balance + amount, "Balance invariant violated"

    def withdraw(self, amount: float) -> None:
        # Preconditions
        assert amount > 0, f"Withdrawal amount must be positive, got {amount}"
        assert amount <= self._balance, (
            f"Insufficient funds: requested {amount}, available {self._balance}"
        )

        old_balance = self._balance
        self._balance -= amount

        # Postconditions
        assert self._balance == old_balance - amount, "Balance invariant violated"
        assert self._balance >= 0, "Balance went negative -- this should be impossible"


# Normal usage
print("=== Normal usage ===")
account = BankAccount(owner="Alice", _balance=100.0)
account.deposit(50.0)
print(f"  After deposit: ${account.balance:.2f}")
account.withdraw(30.0)
print(f"  After withdrawal: ${account.balance:.2f}")

# Precondition violations caught by assertions
print("\n=== Assertion violations ===")

violations: list[tuple[str, object]] = [
    ("Negative deposit", lambda: account.deposit(-10)),
    ("Overdraft", lambda: account.withdraw(999)),
    ("Empty owner", lambda: BankAccount(owner="", _balance=0)),
    ("Negative initial balance", lambda: BankAccount(owner="Bob", _balance=-50)),
]

for name, action in violations:
    try:
        action()  # type: ignore[operator]
        print(f"  {name}: No error (unexpected!)")
    except AssertionError as e:
        print(f"  {name}: AssertionError: {e}")


# Important: assert vs raise for validation
print("\n=== assert vs raise: when to use which ===")
guidelines = [
    ("assert", "Internal invariants, impossible states, developer errors"),
    ("assert", "Can be disabled with python -O (optimized mode)"),
    ("raise", "User input validation, expected error conditions"),
    ("raise", "Always executes, even in optimized mode"),
]
for keyword, description in guidelines:
    print(f"  {keyword:6s} -> {description}")

## timeit: Micro-Benchmarks

The `timeit` module measures the execution time of small code snippets
accurately, accounting for setup costs and running multiple iterations
to reduce noise. This is the right tool for comparing alternative
implementations of the same operation.

In [ ]:
import timeit


# --- String concatenation: join vs += ---
def build_string_concat(n: int) -> str:
    """Build a string using += concatenation."""
    result = ""
    for i in range(n):
        result += str(i) + " "
    return result


def build_string_join(n: int) -> str:
    """Build a string using str.join (preferred)."""
    return " ".join(str(i) for i in range(n))


def build_string_fstring(n: int) -> str:
    """Build a string using list comprehension + join."""
    parts = [str(i) for i in range(n)]
    return " ".join(parts)


# timeit.timeit runs the statement many times and returns total seconds
n = 1000
iterations = 500

print(f"=== String building ({n} items, {iterations} iterations) ===")

t_concat = timeit.timeit(lambda: build_string_concat(n), number=iterations)
t_join = timeit.timeit(lambda: build_string_join(n), number=iterations)
t_listcomp = timeit.timeit(lambda: build_string_fstring(n), number=iterations)

fastest = min(t_concat, t_join, t_listcomp)
print(f"  += concat:     {t_concat:.4f}s  ({t_concat/fastest:.1f}x)")
print(f"  join(genexpr): {t_join:.4f}s  ({t_join/fastest:.1f}x)")
print(f"  join(listcomp):{t_listcomp:.4f}s  ({t_listcomp/fastest:.1f}x)")


# --- List creation: append vs comprehension ---
print(f"\n=== List creation ({n} items, {iterations} iterations) ===")

t_append = timeit.timeit(
    lambda: [None for _ in range(n)] or [(lst := []) or [lst.append(i * 2) for i in range(n)]] and lst,
    number=iterations,
)

def build_list_append(n: int) -> list[int]:
    result: list[int] = []
    for i in range(n):
        result.append(i * 2)
    return result

def build_list_comprehension(n: int) -> list[int]:
    return [i * 2 for i in range(n)]

t_append = timeit.timeit(lambda: build_list_append(n), number=iterations)
t_comp = timeit.timeit(lambda: build_list_comprehension(n), number=iterations)

fastest = min(t_append, t_comp)
print(f"  list.append(): {t_append:.4f}s  ({t_append/fastest:.1f}x)")
print(f"  list comp:     {t_comp:.4f}s  ({t_comp/fastest:.1f}x)")


# --- Dictionary lookup vs if/elif chain ---
print(f"\n=== Dispatch: dict lookup vs if/elif ===")

def dispatch_if(op: str, a: int, b: int) -> int:
    if op == "add":
        return a + b
    elif op == "sub":
        return a - b
    elif op == "mul":
        return a * b
    return 0

dispatch_table = {
    "add": lambda a, b: a + b,
    "sub": lambda a, b: a - b,
    "mul": lambda a, b: a * b,
}

def dispatch_dict(op: str, a: int, b: int) -> int:
    return dispatch_table[op](a, b)

iters = 100_000
t_if = timeit.timeit(lambda: dispatch_if("mul", 10, 20), number=iters)
t_dict = timeit.timeit(lambda: dispatch_dict("mul", 10, 20), number=iters)

fastest = min(t_if, t_dict)
print(f"  if/elif:    {t_if:.4f}s  ({t_if/fastest:.1f}x)")
print(f"  dict lookup:{t_dict:.4f}s  ({t_dict/fastest:.1f}x)")

## cProfile: Function-Level Profiling

`cProfile` profiles an entire program and reports time spent in each
function call. This is the right tool for finding bottlenecks in
real applications. Key columns in the output:

- **ncalls**: Number of times the function was called
- **tottime**: Total time spent IN the function (excluding sub-calls)
- **cumtime**: Cumulative time (including sub-calls)
- **percall**: Time per call (tottime/ncalls or cumtime/ncalls)

In [ ]:
import cProfile
import pstats
import io


def simulate_data_pipeline() -> dict[str, object]:
    """A multi-step pipeline to profile."""
    raw_data = generate_data(10_000)
    cleaned = clean_data(raw_data)
    transformed = transform_data(cleaned)
    summary = summarize_data(transformed)
    return summary


def generate_data(n: int) -> list[dict[str, object]]:
    """Generate synthetic data records."""
    return [
        {"id": i, "value": i * 1.5, "category": f"cat_{i % 5}", "valid": i % 7 != 0}
        for i in range(n)
    ]


def clean_data(records: list[dict[str, object]]) -> list[dict[str, object]]:
    """Filter out invalid records."""
    return [r for r in records if r["valid"]]


def transform_data(records: list[dict[str, object]]) -> list[dict[str, object]]:
    """Apply transformations to each record."""
    results = []
    for record in records:
        transformed = {
            "id": record["id"],
            "value_squared": float(record["value"]) ** 2,  # type: ignore[arg-type]
            "category": str(record["category"]).upper(),
        }
        results.append(transformed)
    return results


def summarize_data(records: list[dict[str, object]]) -> dict[str, object]:
    """Compute summary statistics."""
    values = [float(r["value_squared"]) for r in records]  # type: ignore[arg-type]
    categories: dict[str, int] = {}
    for r in records:
        cat = str(r["category"])
        categories[cat] = categories.get(cat, 0) + 1

    return {
        "count": len(records),
        "mean_value_sq": sum(values) / len(values) if values else 0,
        "max_value_sq": max(values) if values else 0,
        "categories": categories,
    }


# Profile the pipeline
print("=== cProfile output (sorted by cumulative time) ===")
profiler = cProfile.Profile()
profiler.enable()
result = simulate_data_pipeline()
profiler.disable()

# Format the output
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream)
stats.sort_stats("cumulative")
stats.print_stats(15)  # Top 15 functions
print(stream.getvalue())

print(f"Pipeline result: {result['count']} records, mean={result['mean_value_sq']:.1f}")

# You can also sort by different criteria
print("\n=== Top 5 by total time (excluding sub-calls) ===")
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream)
stats.sort_stats("tottime")
stats.print_stats(5)
print(stream.getvalue())

## Memory Considerations

Python objects carry overhead beyond their raw data. `sys.getsizeof` shows
the shallow memory size of an object. For classes, `__slots__` eliminates
the per-instance `__dict__`, significantly reducing memory for many instances.

In [ ]:
import sys


# --- Object sizes ---
print("=== Object memory sizes (sys.getsizeof) ===")
objects: list[tuple[str, object]] = [
    ("int(0)", 0),
    ("int(1)", 1),
    ("int(2**30)", 2**30),
    ("float(1.0)", 1.0),
    ("bool(True)", True),
    ("None", None),
    ("'' (empty str)", ""),
    ("'hello'", "hello"),
    ("'x' * 100", "x" * 100),
    ("[] (empty list)", []),
    ("list(range(10))", list(range(10))),
    ("list(range(100))", list(range(100))),
    ("{} (empty dict)", {}),
    ("dict 10 items", {i: i for i in range(10)}),
    ("set() empty", set()),
    ("tuple() empty", ()),
    ("tuple(range(10))", tuple(range(10))),
]

for label, obj in objects:
    print(f"  {label:25s} -> {sys.getsizeof(obj):>6} bytes")


# --- __slots__ vs regular classes ---
print("\n=== __slots__ memory savings ===")


class PointRegular:
    """Regular class with __dict__."""
    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z


class PointSlots:
    """Slotted class: no __dict__, fixed attributes."""
    __slots__ = ("x", "y", "z")

    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z


regular = PointRegular(1.0, 2.0, 3.0)
slotted = PointSlots(1.0, 2.0, 3.0)

regular_size = sys.getsizeof(regular) + sys.getsizeof(regular.__dict__)
slotted_size = sys.getsizeof(slotted)

print(f"  Regular (instance + __dict__): {regular_size} bytes")
print(f"  Slotted (instance only):       {slotted_size} bytes")
print(f"  Savings per instance:          {regular_size - slotted_size} bytes ({(1 - slotted_size/regular_size)*100:.0f}%)")

# Scale it up: creating many instances
N = 100_000
print(f"\n  For {N:,} instances:")
print(f"    Regular: ~{regular_size * N / 1024 / 1024:.1f} MB")
print(f"    Slotted: ~{slotted_size * N / 1024 / 1024:.1f} MB")

# Slots limitation: cannot add arbitrary attributes
print("\n  Slots limitation:")
try:
    slotted.w = 4.0  # type: ignore[attr-defined]
except AttributeError as e:
    print(f"    Cannot add attribute: {e}")

## Common Performance Patterns

Python has well-known performance characteristics. Understanding these
patterns helps you write efficient code without premature optimization.

Key patterns:
1. **Local variable lookup** is faster than global or attribute lookup
2. **`str.join()`** is faster than `+=` for building strings
3. **Generators** use constant memory; lists use O(n) memory
4. **List comprehensions** are faster than manual `append` loops
5. **`in` on sets/dicts** is O(1); `in` on lists is O(n)

In [ ]:
import timeit
import sys


# --- Local vs global vs attribute lookup ---
print("=== Variable lookup speed ===")

GLOBAL_VALUE = 42

class Container:
    value = 42

container = Container()

def lookup_global() -> int:
    total = 0
    for _ in range(1000):
        total += GLOBAL_VALUE
    return total

def lookup_local() -> int:
    local_value = 42
    total = 0
    for _ in range(1000):
        total += local_value
    return total

def lookup_attribute() -> int:
    total = 0
    for _ in range(1000):
        total += container.value
    return total

iters = 2000
t_global = timeit.timeit(lookup_global, number=iters)
t_local = timeit.timeit(lookup_local, number=iters)
t_attr = timeit.timeit(lookup_attribute, number=iters)

fastest = min(t_global, t_local, t_attr)
print(f"  Global:    {t_global:.4f}s  ({t_global/fastest:.2f}x)")
print(f"  Local:     {t_local:.4f}s  ({t_local/fastest:.2f}x)")
print(f"  Attribute: {t_attr:.4f}s  ({t_attr/fastest:.2f}x)")


# --- Membership testing: list vs set ---
print("\n=== Membership testing: list O(n) vs set O(1) ===")

data_list = list(range(10_000))
data_set = set(data_list)
search_values = [5000, 9999, 10_001]  # Middle, end, missing

for target in search_values:
    iters = 5000
    t_list = timeit.timeit(lambda: target in data_list, number=iters)
    t_set = timeit.timeit(lambda: target in data_set, number=iters)
    print(f"  Search for {target:>6}: list={t_list:.4f}s, set={t_set:.4f}s, ratio={t_list/t_set:.0f}x")


# --- Generator vs list: memory comparison ---
print("\n=== Generator vs list memory ===")

n = 1_000_000

# List stores all values in memory
large_list = [i * 2 for i in range(n)]
list_size = sys.getsizeof(large_list)

# Generator computes values on demand
large_gen = (i * 2 for i in range(n))
gen_size = sys.getsizeof(large_gen)

print(f"  List of {n:,} items: {list_size:>12,} bytes ({list_size / 1024 / 1024:.1f} MB)")
print(f"  Generator:         {gen_size:>12,} bytes")
print(f"  Memory ratio: {list_size / gen_size:,.0f}x")

# Both produce the same sum
print(f"\n  sum(list) == sum(generator): {sum(large_list) == sum(i * 2 for i in range(n))}")


# --- Practical tip: use generators in pipelines ---
print("\n=== Generator pipeline (constant memory) ===")

def read_records(n: int):
    """Simulate reading records from a large file."""
    for i in range(n):
        yield {"id": i, "value": i * 1.5, "active": i % 3 != 0}

def filter_active(records):
    """Filter to active records only."""
    for record in records:
        if record["active"]:
            yield record

def extract_values(records):
    """Extract just the values."""
    for record in records:
        yield record["value"]

# This pipeline processes 1M records in constant memory
pipeline = extract_values(filter_active(read_records(1_000_000)))
total = sum(pipeline)  # Only one record in memory at a time
print(f"  Sum of 1M filtered records: {total:,.1f}")
print(f"  Pipeline generator size: {sys.getsizeof(pipeline)} bytes (constant!)")